In [66]:
#November 2nd, 2022
#Hungarian Algorithm by Sebnem Demirtas.
import numpy as np

In [67]:
#Acts like main function, assignment cost table is the input.
def hungarian(table):
    
    original_table=table.copy() #Keeping original matrix to use in the end.
    
    N = len(table) #Number of iterations depend on the matrix length.
    
    #Step 1, subtracting minimum from rows.
    min_row=table.min(1)
    for i in range(N):
        for j in range(N):
            table[i][j]-=min_row[i]

    #Step 2, subttracting minimum from columns.
    min_col=table.min(0)  
    for j in range(N):
        for i in range(N):
            table[i][j]-=min_col[j]     
    
    #First optimality check after modifications. Check optimality_check function for details.
    optimal, line_matrix = optimality_check(table)
    
    #If after step 1 and 2 not optimal, algorithm enters a loop. Step 4 is done and checked for optimality in the loop.
    #Once it is optimal, loop terminates.
    while optimal==False:
        step4(table,line_matrix) #Check step4 function for details.
        optimal, line_matrix = optimality_check(table) 

    print("Original table:\n", original_table)
    print("Latest version after modifications:\n", table)
    
    #Once optimality condition holds, the algorithm should find the solution. Check solution function for details.
    solutions = solution(table)
    print("Assignments are: ", solutions)
    
    #Calculating objective function value according to the assignments.
    obj=0
    for s in solutions:
        i,j=s
        obj +=original_table[i][j]
    print("Objective function is: ", obj)

In [68]:
#Checks if the current table is optimal by trying to cover 0s with minimum lines. If the number of lines=N, it is optimal.
#Returns if the table is optimal and how the lines are drawn. 
#In lined_mirror matrix, 0 -> no line, 1->line, 2-> intersection of lines.
def optimality_check(table):
    print("OPTIMALITY CHECK RUNNING")
    
    #First setting optimalit to false.
    optimal=False
    N = len(table)
    #Create a mirror matrix that will be used to check where zeros are in the matrix. 0 for 0s in table, 1 for other values.
    mirror_10 = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if table[i][j]!=0:
                mirror_10[i][j]=1
    
    #Creating another matrix that will show the lines.
    lined_mirror = np.zeros((N,N))
    counter = N+1
    line = 0
    
    #print(lined_mirror)
    #print(mirror_10)
    
    #The loop will began from N and go to 0. It will check rows and columns with that many zeros. If it finds such a row or column,
    #it will draw a line. Line will be shown in lined_mirror matrix. To not count the same zeros again, it will convert lined out zeros
    #in mirror_10 to 1.
    
    #Meanwhile, it counts the number of lines.
    
    for num in range(N):
        counter -=1
        for i in range(N):
    
            if np.count_nonzero((mirror_10[i]==0))== counter: #Is there that many zeros in this row?
                lined_mirror[i] +=1
                mirror_10[i] = 1
                line+=1
            
            if np.count_nonzero((mirror_10[:,i]==0))== counter: #Is there that many zeros in this column?
                lined_mirror[:,i]+=1
                mirror_10[:,i] = 1
                line+=1
                
    #If the number of lines drawn is equal to N, it is optimal.
    if line==N:
        optimal=True
    print("Optimality check result: ",optimal)
    return optimal,lined_mirror

In [69]:
#Step4 is finding the minimum among not crossed out cells. Adding that value to interections and subtracting from not crossed outs.
#Input parameters are the curren table and its line_matrix, the that was generated in optimality_check.
def step4(table, line_matrix):
    print("STEP 4 RUNNING")
    N = len(table)
    minimum=9999 
    
    #Finding minimum among not lined cells.
    for i in range(4):
        for j in range(4):
            if line_matrix[i][j]==0:
                if table[i][j]<minimum:
                    minimum = table[i][j]
    
    #Checking the line_matrix to see which cells are intersections and which sells are not lined.
    for i in range(4):
        for j in range(4):
            if line_matrix[i][j]==0:
                table[i][j]-=1
            elif line_matrix[i][j]==2:
                table[i][j]+=1

In [73]:
#Currrent table is the input. This function will make the necessary assignment to the matrix that is known to have an optimal solution.
def solution(table):
    N = len(table)
    #Create a mirror matrix that will hold the places of zero. Similar to the one in optimality_check.
    mirror_10 = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            if table[i][j]!=0:
                mirror_10[i][j]=1
    #The array that will hold the solution indexes is created.
    indexes = []
    
    #Assignment will be done once every cell in mirror_10 becomes 1. Until then, assignment should go on. So, a loop is created.
    while mirror_10.all()!=1:
        #Assignment should start with the rows and columns with the lowest number of zeros. There is a loop to go from 1 to N.
        for num in range(N):
            #We will check if there are "counter" many 0s.
            counter=num+1
            
            #We should make sure to go through the matrix at least N times because after an assignment, 0 number may decrease in another row.
            #In order to catch it, algorithm loops N times.
            for x in range(N):
                #Loops through the rows.
                for i in range(N):
                    
                    #If number of 0s is equal to the counter,
                    if np.count_nonzero((mirror_10[i]==0))== counter:
                        #It gets the second index of zeros.
                        j = np.where(mirror_10[i] == 0)
                        #Assigns i and the first element in the j array (there might be multiple 0s) to indexes list.
                        indexes.append((i,int(np.array(j[0][0]))))
                        
                        #To make sure there will not be any assignment to the same column or row, mirror_10 matrix's related
                        #row and columns are set to 1.
                        mirror_10[i] = 1
                        mirror_10[:,j[0][0]] = 1

    return indexes

In [74]:
table1 = np.array([[1,2,3,4,5],
                 [2,4,1,5,3],
                 [5,2,3,1,4],
                 [1,3,5,2,4],
                 [4,5,3,2,1]])

table2=np.array([[1,4,6,3],
                 [9,7,10,9],
                 [4,5,11,7],
                 [8,7,8,5]])

SOLUTIONS:

In [75]:
hungarian(table1)

OPTIMALITY CHECK RUNNING
Optimality check result:  True
Original table:
 [[1 2 3 4 5]
 [2 4 1 5 3]
 [5 2 3 1 4]
 [1 3 5 2 4]
 [4 5 3 2 1]]
Latest version after modifications:
 [[0 0 2 3 4]
 [1 2 0 4 2]
 [4 0 2 0 3]
 [0 1 4 1 3]
 [3 3 2 1 0]]
Assignments are:  [(1, 2), (3, 0), (4, 4), (0, 1), (2, 3)]
Objective function is:  6


In [76]:
hungarian(table2)

OPTIMALITY CHECK RUNNING
Optimality check result:  False
STEP 4 RUNNING
OPTIMALITY CHECK RUNNING
Optimality check result:  True
Original table:
 [[ 1  4  6  3]
 [ 9  7 10  9]
 [ 4  5 11  7]
 [ 8  7  8  5]]
Latest version after modifications:
 [[0 2 1 1]
 [3 0 0 2]
 [0 0 3 2]
 [4 2 0 0]]
Assignments are:  [(0, 0), (2, 1), (1, 2), (3, 3)]
Objective function is:  21
